<a href="https://colab.research.google.com/github/GregoryG3/Thesis/blob/main/3_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary libraries

In [2]:
# !pip install python-docx

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime
from tqdm import tqdm

from google.colab import drive
from os.path import join

import re
from scipy.stats import describe

from matplotlib import pyplot as plt
import seaborn as sns

# from docx import Document
# from docx.shared import Pt
# from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
# from docx.oxml.ns import qn
# from docx.oxml import OxmlElement

# import string
# import nltk
# from nltk.util import ngrams
# from nltk.corpus import stopwords
# from collections import Counter
# from bs4 import BeautifulSoup

# from matplotlib.lines import Line2D
# from wordcloud import WordCloud
# # from PIL import Image

# Load data

In [2]:
drive.mount('/content/drive')
PROJECT_DIR = "/content/drive/MyDrive/Thesis/Data"
FEEDBACK_DIR = join(PROJECT_DIR, "Feedback data")
PROCESSED_DIR = join(PROJECT_DIR, "Processed")

Mounted at /content/drive


Loading the AB feedback dataset with predicted snetiments, topics and motives:

In [3]:
prediction_dataset = pd.read_csv(join(PROCESSED_DIR, "ab_predicted_2.csv"), encoding='latin1')
## finalnie trzeba pobrac plik: ab_predicted_final.csv

Loading the dataset with manually coded
 data and whole Ab_feedback for further processing:



In [4]:
manually_coded = pd.read_csv(join(PROJECT_DIR,"Annotation data/manual_coding_majority_vote_2000_texts.csv"), encoding='latin1')
ab_text_feedback = pd.read_csv(join(FEEDBACK_DIR, "ab_selected_feedback_data.csv"), encoding='latin1')

Loading the AB items

In [3]:
ab_items = pd.read_csv(join(FEEDBACK_DIR,"ab_items.csv"), encoding='latin1')

# Preprocessing

## Preparing full_dataset

Prepare the manually coded data to be joined back in:


In [62]:
# Filter rows where global_id is in manually_coded
manually_coded_extra_info = ab_text_feedback[ab_text_feedback['global_id'].isin(manually_coded['global_id'])]

# Merge tables based on the 'global_id' column
col = ['global_id', 't_communication', 't_payment', 't_refund',
       't_price', 't_value', 't_shipping', 't_product', 't_feedback',
       't_vendor', 't_generic', 't_overall', 'pos_neg',
       'm_help_other_buyer', 'm_avoid_harm_seler', 'm_help_seller',
       'm_rew_pun_seller', 'm_reach_seller', 'm_express_emo', 'm_share_facts']

manually_coded_combi = pd.merge(manually_coded_extra_info.iloc[:, [0, 1, 3, 4, 5, 6, 7, 8]],
                                manually_coded.loc[:, col],
                                on='global_id')

# Tidy up the dates
manually_coded_combi.loc[:, 'date_left'] = pd.to_datetime(manually_coded_combi['date_left'].str.slice(stop=-6), format='%b %d, %Y', errors='coerce').dt.date


Join the prediction and manually coded subset:


In [63]:
prediction_dataset['origin'] = "prediction"
manually_coded_combi['origin'] = "manual"

# Rename the column to 'm_avoid_harm_seller' - we have a typo
manually_coded_combi = manually_coded_combi.rename(columns={'m_avoid_harm_seler': 'm_avoid_harm_seller'})

full_dataset = pd.concat([manually_coded_combi, prediction_dataset], axis=0)

Change type of columns

In [64]:
# Define the columns to convert
cols_to_convert = full_dataset.columns[8:27]

# Function to convert to Int64 if the value is finite, otherwise keep NaN
def to_int_if_finite(series):
    return series.apply(lambda x: int(x) if pd.notna(x) and np.isfinite(x) else pd.NA).astype('Int64')

# Apply the conversion to the specified columns with a progress bar
for col in tqdm(cols_to_convert, desc="Converting columns in full_dataset"):
    full_dataset[col] = to_int_if_finite(full_dataset[col])


Converting columns in full_dataset: 100%|██████████| 19/19 [00:37<00:00,  1.97s/it]


## Preprocessing AB_items

First we need to determine the weight of the products based on the Name column


In [9]:
# # Filter the DataFrame and get unique names
# ab_item_filter = ab_items[ab_items['cat1'] == 'Drugs & Chemicals']['name'].drop_duplicates()

# Sample 1000 unique names
sample_df = pd.DataFrame()
sample_df['name'] = ab_items[ab_items['cat1'] == 'Drugs & Chemicals']['name'].drop_duplicates().sample(n=5, random_state=1)

In [7]:
sample_df.to_csv('sample_unique_names.csv', index=False, header=True)

In [158]:
import pandas as pd
from transformers import pipeline

# Inicjalizacja modelu do ekstrakcji informacji
nlp = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [167]:
from transformers import BertTokenizer, BertForTokenClassification

In [169]:
# Funkcja do wyciągania wagi za pomocą modelu transformacyjnego


# Inicjalizacja modelu i tokenizera BERT
model_name  = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

# Użycie pipeline do ekstrakcji encji
nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Funkcja do konwersji jednostek na gramy
def convert_to_grams(weight, unit):
    unit = unit.lower()
    if unit in ['mg', 'milligram']:
        return weight / 1000  # Przelicz mg na g
    elif unit in ['kg', 'kilogram']:
        return weight * 1000  # Przelicz kg na g
    elif unit in ['lb', 'pound']:
        return weight * 453.592  # Przelicz lb na g
    return weight  # g już jest w gramach

# Funkcja do ekstrakcji wag z opisów
def extract_weight_with_bert(description):
    entities = nlp(description)
    total_weight = 0
    debug_info = {"description": description, "matches": []}

    for entity in entities:
        entity_text = entity['word']
        # Dopasowanie wzorców liczbowych i jednostek w tekście encji
        match = re.search(r"(\d*\.?\d+)\s*(mg|g|kg|lb|milligram|gram|kilogram|pound)", entity_text, re.IGNORECASE)
        if match:
            weight, unit = match.groups()
            weight = float(weight)
            weight_in_grams = convert_to_grams(weight, unit)
            total_weight += weight_in_grams
            debug_info["matches"].append((1, weight_in_grams, unit, "entity"))

    debug_info["total_weight"] = total_weight
    return total_weight if total_weight > 0 else None, debug_info

# Zastosowanie funkcji do kolumny 'name'
tqdm.pandas()
sample_df['weight'] = sample_df['name'].progress_apply(extract_weight_with_bert)

sample_df.head(10)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 50/50 [00:06<00:00,  8.01it/s]


,name,weight
76135,2x White JigSaw 220 MG MDMA NETTO labtested |,"(0.22, {'description': '2x White JigSaw 220 MG MDMA NETTO labtested |', 'matches': [(1, 0.22, 'mg', 'entity')], 'total_weight': 0.22})"
108223,1 lb Trim |,"(453.592, {'description': '1 lb Trim |', 'matches': [(1, 453.592, 'lb', 'entity')], 'total_weight': 453.592})"
66043,8 x Diazepam 10mg Crescent Brand |,"(None, {'description': '8 x Diazepam 10mg Crescent Brand |', 'matches': [], 'total_weight': 0})"
103870,250g diclazepam powder |,"(250.0, {'description': '250g diclazepam powder |', 'matches': [(1, 250.0, 'g', 'entity')], 'total_weight': 250.0})"
100263,400 HIGH QUALITY DOUBLE DOSED 4MG XANAX BARS (CANADA ONLY) |,"(0.004, {'description': '400 HIGH QUALITY DOUBLE DOSED 4MG XANAX BARS (CANADA ONLY) |', 'matches': [(1, 0.004, 'mg', 'entity')], 'total_weight': 0.004})"
82357,10g Indica PREMIUM Weed *SPECIAL OFFER* |,"(10.0, {'description': '10g Indica PREMIUM Weed *SPECIAL OFFER* |', 'matches': [(1, 10.0, 'g', 'entity')], 'total_weight': 10.0})"
133578,PROMOTION peruvian FISHSCALE COCAINE 2 gram 80 Really Clean C TOP NOTCH STUFF !!! |,"(2.0, {'description': 'PROMOTION peruvian FISHSCALE COCAINE 2 gram 80 Really Clean C TOP NOTCH STUFF !!! |', 'matches': [(1, 2.0, 'g', 'entity')], 'total_weight': 2.0})"
30547,1.5 lb/ 24OZ SOUR DIESEL GRADE AA+ |,"(None, {'description': '1.5 lb/ 24OZ SOUR DIESEL GRADE AA+ |', 'matches': [], 'total_weight': 0})"
128465,0.5 g Ketamine S isomer |,"(5.0, {'description': '0.5 g Ketamine S isomer |', 'matches': [(1, 5.0, 'g', 'entity')], 'total_weight': 5.0})"
10205,2g - Moroccan Polm (Sativa) (Escrow) |,"(2.0, {'description': '2g - Moroccan Polm (Sativa) (Escrow) |', 'matches': [(1, 2.0, 'g', 'entity')], 'total_weight': 2.0})"


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
from tqdm import tqdm

In [6]:
# Inicjalizacja modelu i tokenizera GPT-3
model_name = "EleutherAI/gpt-neo-2.7B"  # Możesz użyć innego modelu GPT-3 dostępnego na Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

In [10]:
# Funkcja do konwersji jednostek na gramy
def convert_to_grams(weight, unit):
    unit = unit.lower()
    if unit in ['mg', 'milligram']:
        return weight / 1000  # Przelicz mg na g
    elif unit in ['kg', 'kilogram']:
        return weight * 1000  # Przelicz kg na g
    elif unit in ['lb', 'pound']:
        return weight * 453.592  # Przelicz lb na g
    return weight  # g już jest w gramach

# Funkcja do ekstrakcji wag przy użyciu modelu GPT-3
def extract_weight_gpt(description):
    prompt = f"Extract the total weight in grams from the following product description: '{description}'"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    debug_info = {"description": description, "extracted_text": response}

    # Próbuj parsować wynik jako liczba
    try:
        weight = float(response.split()[0])
    except (ValueError, IndexError):
        weight = None

    return weight, debug_info

# Zastosowanie funkcji do DataFrame
tqdm.pandas()
sample_df['weight'], sample_df['debug_info'] = zip(*sample_df['name'].progress_apply(extract_weight_gpt))
sample_df.head()

100%|██████████| 5/5 [04:07<00:00, 49.55s/it]


,name,weight,debug_info
76135,2x White JigSaw 220 MG MDMA NETTO labtested |,None,{'description': '2x White JigSaw 220 MG MDMA N...
108223,1 lb Trim |,None,"{'description': '1 lb Trim |', 'extracted_text..."
66043,8 x Diazepam 10mg Crescent Brand |,None,{'description': '8 x Diazepam 10mg Crescent Br...
103870,250g diclazepam powder |,None,"{'description': '250g diclazepam powder |', 'e..."
100263,400 HIGH QUALITY DOUBLE DOSED 4MG XANAX BARS (...,None,{'description': '400 HIGH QUALITY DOUBLE DOSED...


In [8]:
sample_df

,name
76135,2x White JigSaw 220 MG MDMA NETTO labtested |
108223,1 lb Trim |
66043,8 x Diazepam 10mg Crescent Brand |
103870,250g diclazepam powder |
100263,400 HIGH QUALITY DOUBLE DOSED 4MG XANAX BARS (...
82357,10g Indica PREMIUM Weed *SPECIAL OFFER* |
133578,PROMOTION peruvian FISHSCALE COCAINE 2 gram ...
30547,1.5 lb/ 24OZ SOUR DIESEL GRADE AA+ |
128465,0.5 g Ketamine S isomer |
10205,2g - Moroccan Polm (Sativa) (Escrow) |


In [133]:
ab_items[ab_items['cat1']== 'Drugs & Chemicals']

,itemid,seller,name,description,class,quantityleft,endsin,shipsfrom,shipsto,payment,price,cat1,cat2,cat3,shipping_is_0,date_le,n_sold,since,vendor_level,trust_level
10,34,pandoraz,Viagra (Sildenafil) 10 x 150mg |,What is Viagra?Viagra is a prescription-only drug used to treat erectile dysfunction (also known as impotence). It is actually the trade name of a medicine called sildenafil citrate and is used to help men gain and maintain an erection sufficient for penetrative sex.If you have no prescription but want a massive erection pill then look no further.,Physical package,100 items,Never,United Kingdom,"North America, United Kingdom",Escrow,17.00,Drugs & Chemicals,Prescription,Prescription,0.0,2017-06-13,2.0,2014-12-22,1.0,3.0
11,35,pandoraz,Viagra (Sildenafil) 20 x 150mg |,What is Viagra?Viagra is a prescription-only drug used to treat erectile dysfunction (also known as impotence). It is actually the trade name of a medicine called sildenafil citrate and is used to help men gain and maintain an erection sufficient for penetrative sex.If you have no prescription but want a massive erection pill then look no further.,Physical package,100 items,Never,United Kingdom,"North America, United Kingdom",Escrow,31.00,Drugs & Chemicals,Prescription,Prescription,0.0,2017-06-13,0.0,2014-12-22,1.0,3.0
12,36,pandoraz,Viagra (Sildenafil) 30 x 150mg |,What is Viagra?Viagra is a prescription-only drug used to treat erectile dysfunction (also known as impotence). It is actually the trade name of a medicine called sildenafil citrate and is used to help men gain and maintain an erection sufficient for penetrative sex.If you have no prescription but want a massive erection pill then look no further.,Physical package,100 items,Never,United Kingdom,"North America, United Kingdom",Escrow,45.00,Drugs & Chemicals,Prescription,Prescription,0.0,2017-06-13,0.0,2014-12-22,1.0,3.0
13,37,pandoraz,Cialis (Tadalafil) 10 x 40mg |,"What is Cialis?Cialis is a popular treatment for erectile dysfunction that can be used by men aged 18 or over. It is a prescription-only tablet that contains the active ingredient tadalafil, and is taken to help induce an erection and allow for sexual intercourse. Similar erectile dysfunction treatments include Viagra (sildenafil)Cialis can be taken:10mg, 20mg or 40mg for episodic, or Â36-hourÂ, use.",Physical package,100 items,Never,United Kingdom,"North America, United Kingdom",Escrow,18.00,Drugs & Chemicals,Prescription,Prescription,0.0,2017-06-13,0.0,2014-12-22,1.0,3.0
14,38,pandoraz,Cialis (Tadalafil) 20 x 40mg |,"What is Cialis?Cialis is a popular treatment for erectile dysfunction that can be used by men aged 18 or over. It is a prescription-only tablet that contains the active ingredient tadalafil, and is taken to help induce an erection and allow for sexual intercourse. Similar erectile dysfunction treatments include Viagra (sildenafil)Cialis can be taken:10mg, 20mg or 40mg for episodic, or Â36-hourÂ, use.",Physical package,100 items,Never,United Kingdom,"Worldwide, United Kingdom",Escrow,34.00,Drugs & Chemicals,Prescription,Prescription,0.0,2017-06-13,0.0,2014-12-22,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139846,378041,InternationalC,**** 2G ***** WHITE NIGHTMARE **** $$ IC $$ |,"Blue DreamÂs soaring sativa effects and fruity flavors collide with White MoonshineÂs resinous coat of milky trichomes to produce White Nightmare. The sativa-dominant hybrid brings a forceful, sweet aroma of berries and Haze. The flavor stays true to its Blue Dream genetics but with an earthy, hash-like aftertaste. White Nightmare generates a stimulating stream of euphoria and bliss that elevates the mood and brings with it a comfortable, relaxing body high.",Physical package,Unlimited,Never,United Kingdom,"Worldwide, South America, Oceania",Escrow,19.08,Drugs & Chemicals,Cannabis & Hashish,Buds & Flowers,0.0,2017-06-30,2.0,2017-06-25,1.0,3.0
139847,378317,hollandexpress2000,0.2 Gram Colombi

In [87]:
full_dataset[full_dataset['category']=='Cannabis & Hashish']['itemid'].value_counts()

itemid
30011.0     2372
86140.0     2002
29955.0     1923
29982.0     1683
111001.0    1563
            ... 
276395.0       1
289132.0       1
305464.0       1
294903.0       1
290230.0       1
Name: count, Length: 16428, dtype: int64

In [82]:
full_dataset['category'].value_counts()

category
Cannabis & Hashish              324481
Stimulants                      202472
Ecstasy                         119763
Opioids                         104885
Psychedelics                     96870
Benzos                           82870
CVV & Cards                      52697
Other                            48471
Prescription                     37667
Accounts & Bank Drops            33121
Dissociatives                    30639
Steroids                         20153
Personal Information & Scans     10830
Fraud                             9015
Money                             7361
Tobacco                           4922
Dumps                             3644
Fake IDs                          3472
Weight Loss                       2840
Paraphernalia                     2033
Legit Software                    1934
Digital                           1717
Carding                           1681
Social Engineering                1579
Game Keys                         1406
Botnets & Malwar

In [103]:
ab_items['shipsto'].value_counts()

shipsto
Worldwide                                                 82090
United States                                             20183
Europe                                                     7692
Australia                                                  5888
United Kingdom                                             4306
                                                          ...  
Belize, Switzerland                                           1
Worldwide, North America, South America, Asia, Germany        1
Europe, France, Germany, Sweden, United Kingdom               1
Austria, Germany, Italy, Netherlands, United Kingdom          1
Worldwide, South America, Oceania                             1
Name: count, Length: 852, dtype: int64

In [110]:
ab_items['class'].value_counts()

class
Physical package    103540
Digital goods        36313
Name: count, dtype: int64

In [114]:
pd.set_option('display.max_colwidth', None)

In [117]:
ab_items[ab_items['itemid'] == 30011]

,itemid,seller,name,description,class,quantityleft,endsin,shipsfrom,shipsto,payment,price,cat1,cat2,cat3,shipping_is_0,date_le,n_sold,since,vendor_level,trust_level
9348,30011,HumboldtFarms,SALE! 1x Top-shelf 500mg CO2 extracted THC clear oil vape cartridge (made from Grade A++++ Winterized Clear Shatter) |,"<U+27A4> Premium Topshelf 0.5mL Vape Cartidges <U+27A4> These cartridges use CLEAR cape oil which is the purest thing you can smoke <U+27A4> 500mg (1/2 gram) of CO2 extracted Winterized Shatter in each cartridge <U+27A4> Made from Nug Run Clear Shatter is of a high 80% THC level! <U+27A4> Standard 510 thread cartridge - fits most e-cig and vaporizer batteries <U+27A4> Available in Fruity Pebbles (Indica) or Sour Jack (Sativa), Please specify when you purchase",Physical package,Unlimited,Never,United States,United States,Escrow,31.0,Drugs & Chemicals,Cannabis & Hashish,Concentrates,0.0,2017-06-14,5972.0,2015-08-22,10.0,10.0


In [116]:
print(ab_items[ab_items['itemid'] == 30011] ['item'])

KeyError: 'item'

In [96]:
full_dataset[full_dataset['itemid'] == 30011.0]

,seller,date_left,comment,itemid,price,rating,category,global_id,t_communication,t_payment,...,t_overall,pos_neg,m_help_other_buyer,m_avoid_harm_seller,m_help_seller,m_rew_pun_seller,m_reach_seller,m_express_emo,m_share_facts,origin
6,HumboldtFarms,2016-11-08,Fast and professional,30011.0,34.50,Positive,Cannabis & Hashish,ao963123,0,0,...,0,3,0,0,0,0,0,0,1,manual
20,HumboldtFarms,2017-02-28,"Tastes great, works great, is great",30011.0,38.00,Positive,Cannabis & Hashish,ao999839,0,0,...,0,3,0,0,0,0,0,0,1,manual
25,HumboldtFarms,2017-05-16,Marked shipped 5/1-delivered 5/15,30011.0,31.00,Neutral,Cannabis & Hashish,ao2200460,0,0,...,0,2,0,0,0,0,0,0,1,manual
27,HumboldtFarms,2017-04-11,Good stuff,30011.0,33.33,Positive,Cannabis & Hashish,ao970219,0,0,...,0,3,0,0,0,0,0,0,0,manual
28,HumboldtFarms,2016-08-10,it tastes so good and hella strong,30011.0,31.00,Positive,Cannabis & Hashish,ao974273,0,0,...,0,3,0,0,0,0,0,0,1,manual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32320,HumboldtFarms,2017-04-01,This has become like a staple of my diet. Lol....,30011.0,31.00,Positive,Cannabis & Hashish,ao980718,0,0,...,<NA>,2,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,prediction
32337,HumboldtFarms,2017-03-22,"Great as always, thanks you!",30011.0,38.00,Positive,Cannabis & Hashish,ao991017,0,0,...,<NA>,2,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,prediction
32350,HumboldtFarms,2016-11-28,Great product and fast delivery,30011.0,34.50,Positive,Cannabis & Hashish,ao960770,0,0,...,<NA>,2,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,prediction
32352,HumboldtFarms,2017-04-11,F'Ed cause they're always on point,30011.0,38.00,Positive,Cannabis & Hashish,ao969878,0,0,...,<NA>,2,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,prediction


# 4 Preparing AB dataset

Merge two dataset:

In [ ]:
# filter only AB dataset
df_predict_filter = df_predict[df_predict['dataset']=='ab']
# remove duplicates from database with items
ab_items_no_duplicates = ab_items.drop_duplicates(subset=['itemid'], keep='first')

# Convert 'itemid' columns to string, handling NaN values
def is_finite(value):
    try:
        return np.isfinite(float(value))
    except (ValueError, TypeError):
        return False

mask_finite = df_predict_filter['itemid'].apply(is_finite)
df_predict_filter.loc[mask_finite, 'itemid'] = (
    df_predict_filter.loc[mask_finite, 'itemid']
    .astype(float)
    .astype(int)
    .astype(str)
)

ab_items_no_duplicates.loc[:,'itemid'] = ab_items_no_duplicates['itemid'].astype(str)

# Merge the entire datasets based on 'itemid'
ab_feedback = pd.merge(df_predict_filter,
                       ab_items_no_duplicates,
                       on=['itemid','seller'],
                       how='left')

ab_feedback = ab_feedback[ab_feedback['symbols_only']=='no']
ab_feedback_for_prediciton = ab_feedback.drop(columns=['symbols_only', 'dataset','buyer', 'cat2'])
ab_feedback_for_prediciton = ab_feedback_for_prediciton.dropna(subset=['category'])

<ipython-input-63-9ecb390f01f4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict_filter.loc[mask_finite, 'itemid'] = (


In [ ]:
# # filter only AB dataset
# df_predict_filter = df_predict[df_predict['dataset']=='ab']
# # remove duplicates from database with items
# # ab_items_no_duplicates = ab_items.drop_duplicates(subset=['itemid'], keep='first')

# # Convert 'itemid' columns to string, handling NaN values
# def is_finite(value):
#     try:
#         return np.isfinite(float(value))
#     except (ValueError, TypeError):
#         return False

# mask_finite = df_predict_filter['itemid'].apply(is_finite)
# df_predict_filter.loc[mask_finite, 'itemid'] = (
#     df_predict_filter.loc[mask_finite, 'itemid']
#     .astype(float)
#     .astype(int)
#     .astype(str)
# )

# ab_items.loc[:,'itemid'] = ab_items_no_duplicates['itemid'].astype(str)

# df_predict_filter['price'] = pd.to_numeric(df_predict_filter['price'], errors='coerce')
# ab_items['price'] = pd.to_numeric(ab_items['price'], errors='coerce')
# # ab_items_no_duplicates.loc[:,'itemid'] = ab_items_no_duplicates['itemid'].astype(str)

# # Merge the entire datasets based on 'itemid'
# ab_feedback = pd.merge(df_predict_filter,
#                       #  ab_items_no_duplicates,
#                        ab_items,
#                        on=['itemid','seller', 'price'],
#                        how='left')

# ab_feedback = ab_feedback[ab_feedback['symbols_only']=='no']
# ab_feedback_for_prediciton = ab_feedback.drop(columns=['symbols_only', 'dataset','buyer', 'cat2'])
# ab_feedback_for_prediciton = ab_feedback_for_prediciton.dropna(subset=['category'])

<ipython-input-60-8141147f3b78>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict_filter.loc[mask_finite, 'itemid'] = (
<ipython-input-60-8141147f3b78>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict_filter['price'] = pd.to_numeric(df_predict_filter['price'], errors='coerce')


In [ ]:
# Define a function to compare two columns, including NaNs
def compare_columns_with_nan(df, col1, col2):
    return df.apply(lambda x: x[col1] == x[col2] or (pd.isna(x[col1]) and pd.isna(x[col2])), axis=1)

# Apply the function to the DataFrame
comparison_with_nan = compare_columns_with_nan(ab_feedback_for_prediciton, 'seller_x', 'seller_y')

comparison_with_nan.value_counts()

KeyError: 'seller_x'

# Save the file

In [67]:
full_dataset.to_csv(join(PROCESSED_DIR, "full_dataset.csv"), index=False)